In [1]:
import numpy as np

from scipy.stats import uniform, randint, loguniform
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from catboost import CatBoostRegressor

from fit_util import *

In [5]:
train_df = load_data()
transformer = DataTransformer()
train_df = transformer.fit_transform(train_df, obj_to_num=False)

y = train_df['SalePrice']
X = train_df.drop(columns='SalePrice', axis=1)

X_train = X.iloc[np.random.choice(np.arange(len(X)), 300, False)]
y_train = y[X_train.index]

cat_features = np.where(X_train.loc[:, X_train.columns.values].dtypes == "object")[0]
to_categorical(X_train)

/mnt/CommonFiles/Programming/mlcourse/task 4/fit_util.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[c] = X[c].astype('category')
/mnt/CommonFiles/Programming/mlcourse/task 4/fit_util.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[c] = X[c].astype('category')
/mnt/CommonFiles/Programming/mlcourse/task 4/fit_util.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [6]:
parameters = {
    "n_estimators": [1, 10, 20, 100, 1000, 5000, 7000, 9000, 10000],
    "learning_rate": uniform(1e-3,  1e-1),
    "depth": [1, 2, 3, 4, 5],
    "l2_leaf_reg": loguniform(10, 40)
    }

model = CatBoostRegressor( task_type="CPU", 
                           logging_level='Silent', 
                           random_seed=0,
                           cat_features=cat_features)

grid_model = RandomizedSearchCV(model, parameters, cv=5, n_jobs=-1)
grid_model.fit(X_train, y_train);

{'depth': 2, 'l2_leaf_reg': 24.083691576022428, 'learning_rate': 0.09374616611150736, 'n_estimators': 1000}


In [7]:
print(grid_model.best_params_)

{'depth': 2, 'l2_leaf_reg': 24.083691576022428, 'learning_rate': 0.09374616611150736, 'n_estimators': 1000}


In [2]:
train_df = load_data()
transformer = DataTransformer()
train_df = transformer.fit_transform(train_df, obj_to_num=False)

y = train_df['SalePrice']
X = train_df.drop(columns='SalePrice', axis=1)


cat_features = np.where(X.loc[:, X.columns.values].dtypes == "object")[0]
to_categorical(X)

params = {'depth': 2, 'l2_leaf_reg': 24.083691576022428, 'learning_rate': 0.09374616611150736, 'n_estimators': 1000}

gs_model = CatBoostRegressor(**params,
                               task_type="CPU", 
                               logging_level='Silent', 
                               random_seed=0,
                               cat_features=cat_features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98987)
gs_model.fit(X_train, y_train);

In [3]:
evaluate(gs_model, X_train, y_train)
evaluate(gs_model, X_test, y_test)

RMSE: 0.09495450350934387
RMSE: 0.12314745042916674


In [4]:
submission(transformer=transformer, gs_model=gs_model, obj_to_num=True)

RMSLE submission: 0.3220493808846147
